https://climada-python.readthedocs.io/en/stable/tutorial/climada_engine_Impact.html

In [ ]:
# Exposure from the module Litpop
# Note that the file gpw_v4_population_count_rev11_2015_30_sec.tif must be downloaded (do not forget to unzip) if
# you want to execute this cell on your computer. If you haven't downloaded it before, please have a look at the section 
# "population count" of the LitPop tutorial.

import numpy as np
import climada
from climada.entity import LitPop

import sys

from importlib.metadata import version

import climada_petals

import warnings # To hide the warnings
warnings.filterwarnings('ignore')
# Maybe this also?
np.warnings = warnings


In [ ]:
print(f"{np.__version__=}")
print(f"python version: {sys.version_info[0]}.{sys.version_info[1]}.{sys.version_info[2]}")

print(version('climada'))
print(version('climada_petals'))

In [ ]:


# Cuba with resolution 10km and financial_mode = income group.
exp_lp = LitPop.from_countries(countries=['CUB'], res_arcsec=300, fin_mode='income_group', reference_year=2015)
exp_lp.check()


In [ ]:

exp_lp.gdf.head()


In [ ]:
# not needed for impact calculations
# visualize the define exposure
exp_lp.plot_raster()
print('\n Raster properties exposures:', exp_lp.meta)

In [ ]:
from climada.hazard import TCTracks, TropCyclone, Centroids

# Load histrocial tropical cyclone tracks from ibtracs over the North Atlantic basin between 2010-2012
ibtracks_na = TCTracks.from_ibtracs_netcdf(provider='usa', basin='NA', year_range=(2010, 2012), correct_pres=True)
print('num tracks hist:', ibtracks_na.size)

ibtracks_na.equal_timestep(0.5)  # Interpolation to make the track smooth and to allow applying calc_perturbed_trajectories
# Add randomly generated tracks using the calc_perturbed_trajectories method (1 per historical track)
ibtracks_na.calc_perturbed_trajectories(nb_synth_tracks=1)
print('num tracks hist+syn:', ibtracks_na.size)

In [ ]:
# not needed for calculations
# visualize tracks
ax = ibtracks_na.plot()
ax.get_legend()._loc = 2

In [ ]:
# Define the centroids from the exposures position
lat = exp_lp.gdf['latitude'].values
lon = exp_lp.gdf['longitude'].values
centrs = Centroids.from_lat_lon(lat, lon)
centrs.check()


In [ ]:
# Using the tracks, compute the windspeed at the location of the centroids
tc = TropCyclone.from_tracks(ibtracks_na, centroids=centrs)
tc.check()

In [ ]:
from climada.entity import ImpactFuncSet, ImpfTropCyclone
# impact function TC
impf_tc = ImpfTropCyclone.from_emanuel_usa()

# add the impact function to an Impact function set
impf_set = ImpactFuncSet([impf_tc])
impf_set.check()

In [ ]:
# Get the hazard type and hazard id
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]
print(f"hazard type: {haz_type}, hazard id: {haz_id}")

In [ ]:
# Exposures: rename column and assign id
exp_lp.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_lp.gdf['impf_' + haz_type] = haz_id
exp_lp.check()
exp_lp.gdf.head()

This next cell is generating an error. Discussed here. 

https://github.com/CLIMADA-project/climada_python/issues/796

In [ ]:
# Compute impact
from climada.engine import ImpactCalc
imp = ImpactCalc(exp_lp, impf_set, tc).impact(save_mat=False)  # Do not save the results geographically resolved (only aggregate values)

In [ ]:
exp_lp.gdf


In [ ]:
print(f"Aggregated average annual impact: {round(imp.aai_agg,0)} $")


In [ ]:
imp.plot_hexbin_eai_exposure(buffer=1);


In [ ]:
# Compute exceedance frequency curve
freq_curve = imp.calc_freq_curve()
freq_curve.plot();


In [ ]:
from datetime import datetime, date
import pandas as pd

#set a harvest date
harvest_DOY=290 #17 October

#loop over all events an check if they happened before or after harvest
event_ids_post_harvest=[]
event_ids_pre_harvest=[]
for event_id in tc.event_id:
            event_date = tc.date[np.where(tc.event_id==event_id)[0][0]]
            day_of_year = event_date - date(datetime.fromordinal(event_date).year, 1, 1).toordinal() + 1
            
            if day_of_year > harvest_DOY:
                event_ids_post_harvest.append(event_id)
            else:
                event_ids_pre_harvest.append(event_id)

tc_post_harvest=tc.select(event_id=event_ids_post_harvest)
tc_pre_harvest=tc.select(event_id=event_ids_pre_harvest)
#print('pre-harvest:', tc_pre_harvest.event_name)
#print('post-harvest:', tc_post_harvest.event_name)


In [ ]:
from climada.engine import Impact
# impact function TC
impf_tc = ImpfTropCyclone.from_emanuel_usa()
# impact function TC after harvest is by factor 0.5 smaller
impf_tc_posth = ImpfTropCyclone.from_emanuel_usa()
impf_tc_posth.mdd = impf_tc.mdd*0.1
# add the impact function to an Impact function set
impf_set = ImpactFuncSet([impf_tc])
impf_set_posth = ImpactFuncSet([impf_tc_posth])
impf_set.check()
impf_set_posth.check()

#plot
impf_set.plot()
impf_set_posth.plot()

# Compute impacts
imp_preh = ImpactCalc(exp_lp, impf_set, tc_pre_harvest).impact(save_mat=True)
imp_posth = ImpactCalc(exp_lp, impf_set_posth, tc_post_harvest).impact(save_mat=True)

In [ ]:
# Concatenate impacts again
imp_tot = Impact.concat([imp_preh,imp_posth])

#plot result
import matplotlib.pyplot as plt
ax=imp_preh.plot_hexbin_eai_exposure(gridsize=100,adapt_fontsize=False)
ax.set_title('Expected annual impact: Pre-Harvest')
ax=imp_posth.plot_hexbin_eai_exposure(gridsize=100,adapt_fontsize=False)
ax.set_title('Expected annual impact: Post-Harvest')
ax=imp_tot.plot_hexbin_eai_exposure(gridsize=100,adapt_fontsize=False)
ax.set_title('Expected annual impact: Total')

In [ ]:
# EXAMPLE: POINT EXPOSURES WITH POINT HAZARD
import numpy as np
from climada.entity import Exposures, ImpactFuncSet, IFTropCyclone
from climada.hazard import Centroids, TCTracks, TropCyclone
from climada.engine import ImpactCalc

# Set Exposures in points
exp_pnt = Exposures(crs='epsg:4326') #set coordinate system
exp_pnt.gdf['latitude'] = np.array([21.899326, 21.960728, 22.220574, 22.298390, 21.787977, 21.787977, 21.981732])
exp_pnt.gdf['longitude'] = np.array([88.307422, 88.565362, 88.378337, 87.806356, 88.348835, 88.348835, 89.246521])
exp_pnt.gdf['value'] = np.array([1.0e5, 1.2e5, 1.1e5, 1.1e5, 2.0e5, 2.5e5, 0.5e5])
exp_pnt.check()
exp_pnt.plot_scatter(buffer=0.05)

# Set Hazard in Exposures points
# set centroids from exposures coordinates
centr_pnt = Centroids.from_lat_lon(exp_pnt.gdf.latitude.values, exp_pnt.gdf.longitude.values, exp_pnt.crs)
# compute Hazard in that centroids
tr_pnt = TCTracks.from_ibtracs_netcdf(storm_id='2007314N10093')
tc_pnt = TropCyclone.from_tracks(tr_pnt, centroids=centr_pnt)
tc_pnt.check()
ax_pnt = tc_pnt.centroids.plot(c=np.array(tc_pnt.intensity[0,:].todense()).squeeze()) # plot intensity per point
ax_pnt.get_figure().colorbar(ax_pnt.collections[0], fraction=0.0175, pad=0.02).set_label('Intensity (m/s)') # add colorbar

# Set impact function
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_pnt = ImpactFuncSet([impf_tc])
impf_pnt.check()

# Get the hazard type and hazard id
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]
# Exposures: rename column and assign id
exp_lp.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_lp.gdf['impf_' + haz_type] = haz_id
exp_lp.gdf.head()

# Compute Impact
imp_pnt = ImpactCalc(exp_pnt, impf_pnt, tc_pnt).impact()
# nearest neighbor of exposures to centroids gives identity
print('Nearest neighbor hazard.centroids indexes for each exposure:', exp_pnt.gdf.centr_TC.values)
imp_pnt.plot_scatter_eai_exposure(ignore_zero=False, buffer=0.05);

In [ ]:
from rasterio.warp import Resampling
from climada.entity import LitPop, ImpactFuncSet, ImpactFunc
from climada.hazard import Hazard
from climada.engine import Impact
from climada.util.constants import HAZ_DEMO_FL

# Exposures belonging to a raster (the raser information is contained in the meta attribute)
exp_ras = LitPop.from_countries(countries=['VEN'], res_arcsec=300, fin_mode='income_group')
exp_ras.gdf.reset_index()
exp_ras.check()
exp_ras.plot_raster()
print('\n Raster properties exposures:', exp_ras.meta)

# Initialize hazard object with haz_type = 'FL' (for Flood)
hazard_type='FL'
# Load a previously generated (either with CLIMADA or other means) hazard
# from file (HAZ_DEMO_FL) and resample the hazard raster to the exposures' ones
# Hint: check how other resampling methods affect to final impact
haz_ras = Hazard.from_raster([HAZ_DEMO_FL], haz_type=hazard_type, dst_crs=exp_ras.meta['crs'], transform=exp_ras.meta['transform'],
                             width=exp_ras.meta['width'], height=exp_ras.meta['height'],
                             resampling=Resampling.nearest)
haz_ras.intensity[haz_ras.intensity==-9999] = 0 # correct no data values
haz_ras.check()
haz_ras.plot_intensity(1)
print('Raster properties centroids:', haz_ras.centroids.meta)

# Set dummy impact function
intensity = np.linspace(0, 10, 100)
mdd = np.linspace(0, 10, 100)
paa = np.ones(intensity.size)
impf_dum = ImpactFunc(hazard_type, haz_id, intensity, mdd, paa, "m", "dummy")
# Add the impact function to the impact function set
impf_ras = ImpactFuncSet([impf_dum])
impf_ras.check()

# Exposures: rename column and assign id
exp_lp.gdf.rename(columns={"impf_": "impf_" + hazard_type}, inplace=True)
exp_lp.gdf['impf_' + haz_type] = haz_id
exp_lp.gdf.head()

# Compute impact
imp_ras = ImpactCalc(exp_ras, impf_ras, haz_ras).impact(save_mat=False)
# nearest neighbor of exposures to centroids is not identity because litpop does not contain data outside the country polygon
print('\n Nearest neighbor hazard.centroids indexes for each exposure:', exp_ras.gdf.centr_FL.values)
imp_ras.plot_raster_eai_exposure();

In [ ]:
imp_ras

In [ ]:
#imp_pnt.plot_basemap_eai_exposure(buffer=5000);
imp_ras.plot_basemap_eai_exposure(buffer=5000);


In [ ]:
from climada.entity import add_sea
from climada_petals.entity import BlackMarble

exp_video = BlackMarble()
exp_video.set_countries(['Cuba'], 2016, res_km=2.5)
exp_video.check()

# impact function
impf_def = ImpfTropCyclone.from_emanuel_usa()
impfs_video = ImpactFuncSet([impf_def])
impfs_video.check()

# compute sequence of hazards using TropCyclone video_intensity method
exp_sea = add_sea(exp_video, (100, 5))
centr_video = Centroids.from_lat_lon(exp_sea.gdf.latitude.values, exp_sea.gdf.longitude.values)
centr_video.check()

track_name = '2017242N16333'
tr_irma = TCTracks.from_ibtracs_netcdf(provider='usa', storm_id=track_name) # IRMA 2017

tc_video = TropCyclone()
tc_list, _ = tc_video.video_intensity(track_name, tr_irma, centr_video) # empty file name to not to write the video

# generate video of impacts
file_name='./results/irma_imp_fl.gif'
imp_video = Impact()
imp_list = imp_video.video_direct_impact(exp_video, impfs_video, tc_list, file_name)